# Сборный проект


# Описание проекта

Вы работаете в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи вашего мобильного приложения.

Изучите воронку продаж. Узнайте, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?

После этого исследуйте результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясните, какой шрифт лучше.

Создание двух групп A вместо одной имеет определённые преимущества. Если две контрольные группы окажутся равны, вы можете быть уверены в точности проведенного тестирования. Если же между значениями A и A будут существенные различия, это поможет обнаружить факторы, которые привели к искажению результатов. Сравнение контрольных групп также помогает понять, сколько времени и данных потребуется для дальнейших тестов.

В случае общей аналитики и A/A/B-эксперимента работайте с одними и теми же данными. В реальных проектах всегда идут эксперименты. Аналитики исследуют качество работы приложения по общим данным, не учитывая принадлежность пользователей к экспериментам.

# Описание данных

Каждая запись в логе — это действие пользователя, или событие.
- EventName — название события;
- DeviceIDHash — уникальный идентификатор пользователя;
- EventTimestamp — время события;
- ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

### Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
import math
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go 

df = pd.read_csv('/datasets/logs_exp.csv', sep = '\t')

display(df.head())

print(df.info())

### Подготовьте данные

- Замените названия столбцов на удобные для вас;

In [ ]:
df.columns = ['event','user','tmstmp','group']
display(df.head())

- Проверьте пропуски и типы данных. Откорректируйте, если нужно;

In [ ]:
df['tmstmp'] = pd.to_datetime(df['tmstmp'],unit='s')

df['group'] = df['group'].astype('str')

df['group'] = df['group'].replace('246', 'A1')
df['group'] = df['group'].replace('247', 'A2')
df['group'] = df['group'].replace('248', 'B')

display(df.head())

Кроме изменения типа данных, я так же поменял название групп на более легкие к восприятию.

- Добавьте столбец даты и времени, а также отдельный столбец дат;

In [ ]:
df['date'] = df['tmstmp'].dt.date
display(df.head())

In [ ]:
print('Duplicated records: {}'.format(df.duplicated().sum()))
print()

413 дубликатов. Давайте избавимся от них.

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
print('Duplicated records: {}'.format(df.duplicated().sum()))
print()

### Изучите и проверьте данные

- Сколько всего событий в логе?
- Сколько всего пользователей в логе?
- Сколько в среднем событий приходится на пользователя?

In [ ]:
print('Кол-во событии в логе: {}'.format(len(df['event'])))
print('Кол-во пользователей в логе: {}'.format(df['user'].nunique()))
print('В среднем на одного пользователя приходится {:.0f} события'.format(
    (df
         .groupby('user')
         .agg({'event':'count'})
         .reset_index()
    )['event'].mean()
))

- Данными за какой период вы располагаете? Найдите максимальную и минимальную дату. Постройте гистограмму по дате и времени. Можно ли быть уверенным, что у вас одинаково полные данные за весь период? Технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Определите, с какого момента данные полные и отбросьте более старые. Данными за какой период времени вы располагаете на самом деле?
- Много ли событий и пользователей вы потеряли, отбросив старые данные?

In [ ]:
print(
    'Датафрейм содержит информацию с {} по {}'
    .format(
        df['tmstmp'].min(),
        df['tmstmp'].max()
    )
)

In [ ]:
fig = (
    px.histogram(df, x = 'tmstmp')
    .update_layout(
        title = 'Распределение кол-ва событии по датам',
        yaxis_title = 'Кол-во событии',
        xaxis_title = 'Дата'
    )
)

fig.show()

Учитывая показания гистограммы, можно сделать вывод, что в фрейме содержиться полная информация лишь по 7 дням (с 1 по 7 августа). Отбросим ненужные данные, и посмотрим, как много пользователей и событии отсеется.

In [ ]:
limit = datetime.strptime('2019-08-01 00:00:00', '%Y-%m-%d %H:%M:%S')

df_clean = df.query('tmstmp >= @limit')

print(
    'Обновленный датафрейм содержит информацию с {} по {}'
    .format(
        df_clean['tmstmp'].min(),
        df_clean['tmstmp'].max()
    )
)
print('Доля отсеянных событии: {:.2%} ({} событии)'.format(
    (len(df['event']) - len(df_clean['event'])) / len(df['event']),
    len(df['event']) - len(df_clean['event'])
))
print('Доля отсеянных пользователей: {:.2%} ({} пользователей)'.format(
    (df['user'].nunique() - df_clean['user'].nunique()) / df['user'].nunique(),
    df['user'].nunique() - df_clean['user'].nunique()
))

- Проверьте, что у вас есть пользователи из всех трёх экспериментальных групп.

In [ ]:
(
    px.histogram(
        df_clean.groupby('user').agg({'group':'max'}),
        x = 'group'
    )
    .update_layout(
        title = 'Распределение пользователей по группам',
        yaxis_title = 'Кол-во пользователей',
        xaxis_title = 'Группа' 
    )
)

Дополнено: Отлично! Наш датафрейм содержит данные по всем трем группам. Кол-во пользователей при этом примерно одинаково во всех группах (±2500).

### Изучите воронку событий
- Посмотрите, какие события есть в логах, как часто они встречаются. Отсортируйте события по частоте.

In [ ]:
(
    px.histogram(
        df_clean,
        x = 'event'
    )
    .update_xaxes(categoryorder='total descending')
    .update_layout(
        title = 'Распределение событии',
        xaxis_title = 'Событие',
        yaxis_title = 'Кол-во'
    )
)

Дополнено: Похоже, что самым частотным событием является появление пользователя на главном экране. Логично! Самым непопулярным при этом является прохождение обучение. Видимо, оно не особо активно навязывается пользователю, вот все его и игнорируют.

- Посчитайте, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитайте долю пользователей, которые хоть раз совершали событие.

In [ ]:
(
    px.bar(
        df_clean.groupby('event').agg({'user':'nunique'}).reset_index(),
        x = 'event',
        y = 'user'
    )
    .update_xaxes(categoryorder='total descending')
    .update_layout(
        title = 'Распределение событии по кол-ву пользователей',
        xaxis_title = 'Событие',
        yaxis_title = 'Кол-во пользователей'
    )
)

In [ ]:
df_var = df_clean.groupby('event').agg({'user':'nunique'}).reset_index()

df_var['%'] = df_var['user'] / 7534

(
    px.bar(
        df_var,
        x = 'event',
        y = '%'
    )
    .update_xaxes(categoryorder='total descending')
    .update_layout(
        title = '% пользователей совершивших событие',
        xaxis_title = 'Событие',
        yaxis_title = '% пользователей'
    )
)

Дополнено: Чтож, если рассматривать частотность событии в разрезе кол-ва уникальных пользователей, то ситуация будет соотносится с прошлым наблюдением.

- Предположите, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки.

Не до конца понятно, что из себя представляет обучение, и на каком этапе оно должно происходить. Полагаю, что будет логичнее оставить этот этап за скобками.

- По воронке событий посчитайте, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C посчитайте отношение числа пользователей с событием B к количеству пользователей с событием A, а также отношение числа пользователей с событием C к количеству пользователей с событием B.

In [ ]:
df_clean = df_clean.query('event != "Tutorial"')

pre_funnel = (
    df.pivot_table(
        index = 'user',
        columns = 'event',
        values = 'tmstmp',
        aggfunc = 'min'
    )
    .reset_index()
)

display(pre_funnel.head())

Мы создали фрейм для того, чтобы понять, какой путь прошел каждый пользователь. Следующим шагом мы построим воронку с учетом последовательности действии пользователя.

In [ ]:
step_1 = ~pre_funnel['MainScreenAppear'].isna()
step_2 = step_1 & (pre_funnel['OffersScreenAppear'] > pre_funnel['MainScreenAppear'])
step_3 = step_2 & (pre_funnel['CartScreenAppear'] > pre_funnel['OffersScreenAppear'])
step_4 = step_3 & (pre_funnel['PaymentScreenSuccessful'] > pre_funnel['CartScreenAppear'])

d = {
    'step' : ['MainScreenAppear','OffersScreenAppear','CartScreenAppear','PaymentScreenSuccessful'],
    'users' : [
        len(pre_funnel[step_1]['user']),
        len(pre_funnel[step_2]['user']),
        len(pre_funnel[step_3]['user']),
        len(pre_funnel[step_4]['user'])
    ]
}

funnel = pd.DataFrame(data = d)

fig = go.Figure(go.Funnel(y = funnel['step'], x = funnel['users'])) 

fig.show()

Дополнено: Чтож, если смотреть на нашу воронку с учетом предполагаемой последовательности действии, то конверсия составляет 6%! Что звучит наверное и не так уж и плохо, я полагаю (но, конечно, нет пределов совершентсву). 

Дополнено: Интересно, а есть ли пользователи, которые пошли иным путем и не совершали действия в предполагаемой нами последовательности? Давайте взглянем, есть ли такие пользователи, а также как будет выглядеть наша воронка в таком случае.

In [ ]:
display(pre_funnel.query('(~PaymentScreenSuccessful.isna())'))

Дополнено: Есть практически 3000 пользователей, которые прошли к оплате другим путем. Вполне может быть, что и к другим шагам они могли пройти иначе. Давайте посмотрим на воронку.

In [ ]:
df.query('event != "Tutorial"').groupby('event').agg({'user':'nunique'}).reset_index()

In [ ]:
d = df.query('event != "Tutorial"').groupby('event').agg({'user':'nunique'}).reset_index().sort_values(by = 'user', ascending = False)

funnel = pd.DataFrame(data = d)

fig = go.Figure(go.Funnel(y = funnel['event'], x = funnel['user'])) 

fig.show()

Дополнено: Ничего себе! Наша воронка выглядит совсем иначе, если мы берем в расчет пользователей, которые пришли к оплате иным путем. Возможно наш сайт позволяет оформить покупку в один клик?

- На каком шаге теряете больше всего пользователей?

Скорректированно: Если смотреть на "последовательную" воронку, то самый большой процент пользователей мы теряем на шаге перехода к оплате - лишь 28% пользователей оплачивают товар, добавив его в корзину. Возможно, стоит направить эту информацию коллегам для более глубокого анализа.

Кроме этого, очевидно, что есть большое (причем весьма) кол-во пользователей, которые не следуют последовательности, которая заложена в процессе взаимодействия с нашим сайтом. Если смотреть на "последовательную" воронку, то самое большое кол-во пользователей теряется при переходе с главного экрана на экран с предложениями.

- Какая доля пользователей доходит от первого события до оплаты?

6 процентов

### Изучите результаты эксперимента

- Сколько пользователей в каждой экспериментальной группе?

In [ ]:
display(df_clean.groupby('group').agg({'user':'nunique'}))

- Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.

Для проведения необходимых статистических тестов нам нужно понять, сколько пользователей в каждой группе совершили целевое действие.

In [ ]:
test_df = (
    df_clean.pivot_table(
        index = 'group',
        columns = 'event',
        values = 'user',
        aggfunc = 'nunique'
    )
)

display(test_df)

Теперь проведем тест, который позволит нам проверить следующие гипотезы:
- Н0 - Доли двух выборок не различаются
- Н1 - Между долями двух выборок есть разница

Здесь и далее по отношению к каждым группам мы будем проверять этот набор гипотез.

Возьмем в качестве уровня значимости 0.05

In [ ]:
alpha = .05

def share_test(A, B):
    
    p1 = A[0]/B[0]
    
    p2 = A[1]/B[1]
    
    p_combined = (A[0] + A[1]) / (B[0] + B[1])
    
    difference = p1 - p2

    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1/B[0] + 1/B[1]))

    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('Доля пользователей совершивших событие в 1-й группе {:.2%}'.format(A[0]/B[0]))
    
    print('Доля пользователей совершивших событие в 2-й группе {:.2%}'.format(A[1]/B[1]))
    
    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли в группах разными'
        )

# A - пользователи совершившие целевое действие
        
# B - попытки
    
A = [1200,1158]


B = [2450,2476]

share_test(A,B)

In [ ]:
df_evnt = (
    df_clean.pivot_table(
            index = 'group',
            columns = 'event',
            values = 'user',
            aggfunc = 'nunique'
        )
    .reset_index()
    .merge(df_clean.groupby('group').agg({'user':'nunique'}).reset_index(),
           on = 'group',
           how = 'outer'
          )
)

display(df_evnt)

In [ ]:
# 0 - группа А1, 1 - группа А2, 2 - группа Б

def share_test(GrpI, GrpII, step):
    
    p1 = df_evnt.loc[GrpI,step]/df_evnt.loc[GrpI,'user']
    
    p2 = df_evnt.loc[GrpII,step]/df_evnt.loc[GrpII,'user']
    
    p_combined = (df_evnt.loc[GrpI,step] + df_evnt.loc[GrpII,step]) / (df_evnt.loc[GrpI,'user'] + df_evnt.loc[GrpII,'user'])
    
    difference = p1 - p2

    z_value = difference / math.sqrt(p_combined * (1 - p_combined) * (1/B[0] + 1/B[1]))

    distr = st.norm(0, 1)

    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('Доля пользователей совершивших событие {} в 1-й группе {:.2%}'.format(step, A[0]/B[0]))
    
    print('Доля пользователей совершивших событие {} в 2-й группе {:.2%}'.format(step, A[1]/B[1]))
    
    print('p-значение: ', p_value)

    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли в группах разными'
        )
        
share_test(0,1,'PaymentScreenSuccessful')

- Выберите самое популярное событие. Посчитайте число пользователей, совершивших это событие в каждой из контрольных групп. Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?

Самое популярное действие - появление на начальном экране. Давайте узнаем, какая доля пользователей совершила его, а также есть ли различие в долях контрольных групп.

- Н0 - Доли двух выборок не различаются
- Н1 - Между долями двух выборок есть разница


In [ ]:
share_test(0,1,'MainScreenAppear')

Разницы между контрольными группами по отношению к появлениб на начальном экране нет. Давайте проверим остальные события.

In [ ]:
share_test(0,1,'OffersScreenAppear')

print()

share_test(0,1,'CartScreenAppear')

Различии между контрольными группами нет, так что у нас нет основании полагать, что разбиение пользователей между ними произошло некорректно.

- Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?

Теперт нам нужно проверить не один, а два набора гипотез. Таким образом, для избежания ошибки при множественном сравнения мы воспользуемся поправкой Бонферрони для того, чтобы скорректировать значение alpha.

In [ ]:
alpha = .025

print('A1 / B')

print()

share_test(0,2,'MainScreenAppear')

print()

print('A2 / B')

print()

share_test(1,2,'MainScreenAppear')

In [ ]:
print('A1 / B')

print()

share_test(0,2,'OffersScreenAppear')

print()

print('A2 / B')

print()

share_test(1,2,'OffersScreenAppear')

In [ ]:
print('A1 / B')

print()

share_test(0,2,'CartScreenAppear')

print()

print('A2 / B')

print()

share_test(1,2,'CartScreenAppear')

In [ ]:
print('A1 / B')

print()

share_test(0,2,'PaymentScreenSuccessful')

print()

print('A2 / B')

print()

share_test(1,2,'PaymentScreenSuccessful')

In [ ]:
combined = {'group':'A1/A2','CartScreenAppear':2504,'MainScreenAppear':4926,'OffersScreenAppear':3062,'PaymentScreenSuccessful':2358,'user':4995}

df_evnt = df_evnt.append(combined,ignore_index=True)

display(df_evnt)

In [ ]:
print('A Combined / B')

print()

share_test(0,3,'MainScreenAppear')

print()

share_test(0,3,'OffersScreenAppear')

print()

share_test(0,3,'CartScreenAppear')

print()

share_test(0,3,'PaymentScreenSuccessful')

Разницы между объединенными контрольными группами и измененной так же нет.

- Какой уровень значимости вы выбрали при проверке статистических гипотез выше? Посчитайте, сколько проверок статистических гипотез вы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если вы хотите изменить его, проделайте предыдущие пункты и проверьте свои выводы.

Мы использовали скорректированный показатель статистической значимости, но вы только посмотрите на значения P при проверке каждой из гипотез. Очевидно, что оно настолько большое, что очевидно, что изменение уровня значимости не повляиет на результат эксперимента. 

На самом деле, результат кажется достаточно логичным. Тяжело представить, насколько сильно надо было изменить шрифты (или насколько они были ужасны), чтобы сильно повлиять на конверсию.